# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [1]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker

!pip install s3fs
!pip install smdebug"""

'!pip install mlflow\n!pip install pytorch-lightning\n!pip install transformers\n!pip install tqdm\n!pip install sagemaker\n\n!pip install s3fs\n!pip install smdebug'

In [2]:
import sys
sys.path.append('../../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from typing import Any, Dict, Optional

In [3]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import pandas as pd

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [4]:
from deep.constants import *
from deep.utils import *

In [5]:
%load_ext autoreload
%autoreload 2

# Data

You can use the data you want. We advise the `pandas` format.

In [6]:
DATA_PATH = os.path.join(
    '..', '..', '..', "data", "frameworks_data", 'data_v0.7.1'
)


tot_df = pd.read_csv(os.path.join(DATA_PATH, 'new_columns_train_val.csv')).drop_duplicates()

test_df = pd.read_csv(os.path.join(DATA_PATH, 'new_columns_test_v0.7.1.csv'))

/home/selim/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
minimal_results = {
    'present_prim_tags': 0.8,
    'present_sec_tags': 0.85,
    'sectors': 0.89,
    'pillars_1d': 0.85,
    'pillars_2d': 0.81,
    'subpillars_2d_part1': 0.8, 
   'subpillars_2d_part2': 0.72,
   'subpillars_1d_part1': 0.89,
   'subpillars_1d_part2': 0.85,
   'subpillars_1d_part3': 0.84,
    'gender': 0.84,
    'age': 0.84,
    'specific_needs_groups': 0.79,
    'affected_groups_levels_2_3': 0.99,
    'gender_snorkel': 0.87,
    'subpillars_1d': 0.99,
    'subpillars_2d': 0.99,
    'prim_tags_level1': 0.99
}

In [8]:
columns = ['excerpt', 'entry_id',
           'present_prim_tags',
           'sectors',
           'pillars_2d',
           'pillars_1d',
           'subpillars_2d_part1', 
           'subpillars_2d_part2',
           'subpillars_1d_part1',
           'subpillars_1d_part2',
           'subpillars_1d_part3'
        ]

In [9]:
columns = [
    'excerpt', 'entry_id',
    'present_sec_tags',
    'gender',
    'age',
    'specific_needs_groups',
    'affected_groups_levels_2_3',
    'gender_snorkel'
]

In [10]:
columns = [
    'excerpt', 'entry_id',
    'present_sec_tags',
    'gender',
    'age',
    'specific_needs_groups',
    'gender_snorkel'
]

In [11]:
columns = ['excerpt', 'entry_id',
           'prim_tags_level1',
           'subpillars_2d',
           'subpillars_1d'
        ]

In [12]:

tot_df = tot_df[columns]
test_df = test_df[columns]

In [13]:
tot_df.columns

Index(['excerpt', 'entry_id', 'prim_tags_level1', 'subpillars_2d',
       'subpillars_1d'],
      dtype='object')

## Sagemaker Prep

### Session

Configure SageMaker

In [14]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

### Bucket upload

You need to upload data to an S3 bucket. 




In [15]:
MLFLOW_SERVER

'http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/'

In [16]:
sample = True  # To make the computations faster, sample = True.

if sample:
    tot_df = tot_df.sample(n=100_000)
    
job_name = f"pytorch-{formatted_time()}-all-models"  # change it as you prefer
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name  # Do not change this

train_path = str(input_path / 'train.pickle')
val_path = str(input_path / 'val.pickle')


tot_df.to_pickle(train_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6
test_df.to_pickle(val_path, protocol=4)

### Estimator Definition

In [17]:
# GPU instances

instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [18]:
from sagemaker.pytorch import PyTorch

instance_type='ml.p3.2xlarge'

hyperparameters={
    'tracking_uri': MLFLOW_SERVER,
    'experiment_name': "pl-all-models-experiments",
    'max_len': 512,
    'epochs': 1,
    'model_name': 'microsoft/xtremedistil-l6-h256-uncased',
    'tokenizer_name': 'microsoft/xtremedistil-l6-h256-uncased',
    'output_length': 256,
    'training_names':','.join(columns[2:]),
    "instance_type": instance_type,
    'beta_f1': 0.5,
    'nb_repetitions': 1,
    'run_name': 'test_multitask',
    'min_results': str(minimal_results)
}

estimator = PyTorch(
    entry_point='train_mlflow.py',
    source_dir=str('../../../scripts/training/selim/multiclass-lightning'),
    output_path=str(DEV_BUCKET/'models/'),
    code_location=str(input_path),
    instance_type=instance_type,
    instance_count=1,
    role=role,
    framework_version="1.8",
    py_version="py36",
    hyperparameters = hyperparameters,
    job_name=job_name,
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [19]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [20]:
# Fit the estimator

estimator.fit(fit_arguments, job_name=job_name)

2021-12-03 16:27:00 Starting - Starting the training job...
2021-12-03 16:27:26 Starting - Launching requested ML instancesProfilerReport-1638548818: InProgress
......
2021-12-03 16:28:26 Starting - Preparing the instances for training.........
2021-12-03 16:30:10 Downloading - Downloading input data...
2021-12-03 16:30:29 Training - Downloading the training image...............
2021-12-03 16:33:17 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-12-03 16:33:17,565 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-12-03 16:33:17,588 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-12-03 16:33:20,613 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-12-03 16:33:21,057 sagemaker-training-toolkit INFO     Installing dependencies from 

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sagemaker: filename=sagemaker-2.49.1-py2.py3-none-any.whl size=591916 sha256=aa9f35e3364b8f09b453b3255b5c62f375259328364d53b6d55e7fc8cd213a20
  Stored in directory: /root/.cache/pip/wheels/4c/af/ea/8ff5943a87155df5b184e54474fbf2b59b75e5c172854643c6
  Created wheel for aiobotocore: filename=aiobotocore-2.0.1-py3-none-any.whl size=51978 sha256=62a225bd50580fd2f2bb43e62bb28361ecb227d297fcc6987c149ac231508579
  Stored in directory: /root/.cache/pip/wheels/73/b7/3b/15481dc56d092eba140bebca5486d592b34b90f93b99c46d3b
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=f2ed011170c82ff3b4457c563380ebacbc775f33c987adc975a4045ced655d5e
  Stored in directory: /root/.cache/pip/wheels/e9/7b/aa/e18c983d8236b141f85838ba0f8e4e4ae9bcf7f1e00ff726ec
  Created wheel for databricks-cli: filename=databricks_cli-0.16.2-py3-none-any.whl size=106811 sh

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.1
    Uninstalling numpy-1.19.1:
      Successfully uninstalled numpy-1.19.1
  Attempting uninstall: botocore
    Found existing installation: botocore 1.21.63
    Uninstalling botocore-1.21.63:
      Successfully uninstalled botocore-1.21.63
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.61.2
    Uninstalling tqdm-4.61.2:
      Successfully uninstalled tqdm-4.61.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2021.10.1
    Uninstalling fsspec-2021.10.1:
      Successfully uninstalled fsspec-2021.10.1
  Attempting uninstall: 

[2021-12-03 16:41:43.368 algo-1:80 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-12-03 16:41:43.405 algo-1:80 INFO profiler_config_parser.py:102] User has disabled profiler.
[2021-12-03 16:41:43.406 algo-1:80 INFO json_config.py:91] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2021-12-03 16:41:43.406 algo-1:80 INFO hook.py:201] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tensorboard summaries.
[2021-12-03 16:41:43.407 algo-1:80 INFO hook.py:255] Saving to /opt/ml/output/tensors
[2021-12-03 16:41:43.407 algo-1:80 INFO state_store.py:77] The checkpoint config file /opt/ml/input/config/checkpointconfig.json does not exist.
[2021-12-03 16:41:43.614 algo-1:80 INFO hook.py:594] name:model.l0.embeddings.word_embeddings.weight count_params:7813632
[2021-12-03 16:41:43.614 algo-1:80 INFO hook.py:594] name:model.l0.embeddings.position_embeddings.weight count_params:131072
[2021-12-03 16:41:43.615 algo-1:80 INFO hook.


2021-12-03 16:42:12 Uploading - Uploading generated training model
2021-12-03 16:42:12 Failed - Training job failed
ProfilerReport-1638548818: Stopping


UnexpectedStatusException: Error for Training job pytorch-2021-12-03-17-26-42-804-all-models: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python3.6 train_mlflow.py --beta_f1 0.5 --epochs 1 --experiment_name pl-all-models-experiments --instance_type ml.p3.2xlarge --max_len 512 --min_results {'present_prim_tags': 0.8, 'present_sec_tags': 0.85, 'sectors': 0.89, 'pillars_1d': 0.85, 'pillars_2d': 0.81, 'subpillars_2d_part1': 0.8, 'subpillars_2d_part2': 0.72, 'subpillars_1d_part1': 0.89, 'subpillars_1d_part2': 0.85, 'subpillars_1d_part3': 0.84, 'gender': 0.84, 'age': 0.84, 'specific_needs_groups': 0.79, 'affected_groups_levels_2_3': 0.99, 'gender_snorkel': 0.87, 'subpillars_1d': 0.99, 'subpillars_2d': 0.99, 'prim_tags_level1': 0.99} --model_name microsoft/xtremedistil-l6-h256-uncased --nb_repetitions 1 --output_length 256 --run_name test_multitask --tokenizer_name microsoft/xtremedistil-l6-h256-uncased --tracking_uri http://mlflow-deep-387470f3-1883319727.